In [1]:
fr = open('./data/target_validation.csv', 'r')
lines = fr.readlines()

result = []
import re

def findS(word, tag):
    flag = False
    sentiments = ''
    themes = ''
    for index in range(len(tag)):
        if tag[index]=='S':
            flag = True
            tmp1 = word[index]
            sentiments = sentiments + tmp1 + ';'
            tmp2 = ''
            for i in range(index-1, -1, -1):
                if tag[i]=='T':
                    tmp2 = word[i]
                    tag[i] = 'O'
                    break
            if tmp2 == '':
                tmp2 = 'NULL'
            themes = themes + tmp2 + ';'
    if flag == False:
        sentiments = '$$$'
        themes = '$$$'
    return themes, sentiments
total = []
for line in lines:
    tmp = line.strip().split(' ')
#     print(tmp)
    word = []
    tag = []
    for t in tmp:
        tag.append(t[-1])
        word.append(t[:-2])
    res1, res2 = findS(word, tag)
    total.append([res1, res2])

In [2]:
import pandas as pd
total = pd.DataFrame(total)

In [3]:
total

,0,1
0,牙刷;,好用;
1,$$$,$$$
2,NULL;NULL;,坏;好;
3,$$$,$$$
4,$$$,$$$
5,质量;NULL;感觉;,很好;舒服;小贵;
6,速度;,很快;
7,$$$,$$$
8,质量;,还行;
9,NULL;,实用;


In [4]:
import re
import pandas as pd
not_seg = pd.read_csv('./data/validation_content.csv', header=None)
not_cuts = re.compile(u'[。，、？！\.\?,!]')

index_to_index = {}
start = 0
for index, row in not_seg.iterrows():
    cnt = 0
    for s in not_cuts.finditer(row[0] + '。'):
        index_to_index[start + cnt] = index
        cnt = cnt + 1
    start = start + cnt


In [5]:
index_to_index[100930]

19999

In [5]:
merge_sentiment = {}
merge_theme = {}
for index, row in total.iterrows():
    if merge_sentiment.get(index_to_index[index], -1):
        if row[0] == "$$$":
            merge_sentiment[index_to_index[index]] = ""
            merge_theme[index_to_index[index]] = ""
        else:
            merge_sentiment[index_to_index[index]] = row[1]
            merge_theme[index_to_index[index]] = row[0]
    else:
        if row[0] == "$$$":
            pass
        else:
            merge_sentiment[index_to_index[index]] = merge_sentiment[index_to_index[index]] + row[1]
            merge_theme[index_to_index[index]] = merge_theme[index_to_index[index]] + row[0]

In [8]:
import numpy as np
sentiment_array = pd.DataFrame(np.array(list(merge_sentiment.values())))

# sentiment_array.to_csv('validation_sentiment.csv', header=None, index=None, sep=',')
theme_array = pd.DataFrame(np.array(list(merge_theme.values())))

In [9]:
theme_array.to_csv("validation_theme.csv", header=None, index=None, sep='\t')
sentiment_array.to_csv("sentiment_array.csv", header=None, index=None, sep='\t')